In [28]:

from tensorflow import keras
import pandas as pd
from sklearn.model_selection import train_test_split

In [29]:

data = pd.read_csv("./Training_data.csv")[["ID", "KDA", "WinRate"]]

data["Predictions"] = [0 for i in range(len(data["ID"]))]
pred_grid = pd.read_csv("./Prediction_Grid.csv").drop("Unnamed: 0", axis = 1)
pred_grid["Predictions"] = [0 for i in range(len(pred_grid.ID))]
print(len(data.ID)) # Should be 17940
pred_grid.head()
data.head()

17940


,ID,KDA,WinRate,Predictions
0,Aatrox_Top,1.77,0.4768,0
1,Ahri_Mid,2.58,0.4950,0
2,Akali_Mid,2.37,0.4841,0
3,Akali_Top,2.00,0.4592,0
4,Akshan_Mid,2.26,0.5162,0


In [30]:
for id in set(data["ID"]): #Trains a model for each ID, and saves predictions
    pred_data = pred_grid.loc[pred_grid.ID == id].drop(["ID", "Predictions"], axis = 1)
    temp_data = data.loc[data["ID"] == id].drop(["ID", "Predictions"], axis = 1)
    labels = temp_data.pop("WinRate")

    model = keras.Sequential()

    model.add(keras.layers.Dense(units = 256, activation = 'relu'))
    model.add(keras.layers.Dense(units = 128, activation = 'relu'))
    model.add(keras.layers.Dense(units = 128, activation = 'relu'))
    model.add(keras.layers.Dense(units = 64, activation = 'relu'))
    model.add(keras.layers.Dense(units = 32, activation = 'relu'))
    model.add(keras.layers.Dense(units = 1, activation = 'linear'))

    model.compile(optimizer = "adam",
                loss = "mse", #I only care ab MSE bc it is to be used for p-val testing with other MSE,
                metrics = "mae") 

    model.fit(temp_data, 
          labels,
          batch_size = 32, epochs=24,
          verbose=0)
    data.loc[data["ID"] == id, "Predictions"] = model.predict(temp_data, verbose= 0)
    pred_grid.loc[pred_grid.ID == id, "Predictions"] = model.predict(pred_data, verbose = 0)



In [31]:
data["MSE"] = (data.WinRate - data.Predictions)**2 / 92
data.groupby("ID").agg('sum')

,KDA,WinRate,Predictions,MSE
ID,,,,
Aatrox_Top,169.19,45.0314,45.219429,0.000139
Ahri_Mid,243.19,46.4540,46.879507,0.000121
Akali_Mid,217.48,44.7192,44.800476,0.000125
Akali_Top,188.36,43.8457,43.530994,0.000165
Akshan_Mid,204.37,47.1294,46.569035,0.000194
...,...,...,...,...
Ziggs_Adc,230.19,48.0082,47.934303,0.000204
Ziggs_Mid,217.89,45.8509,46.439255,0.000175
Zilean_Support,278.25,46.2686,46.295461,0.000306


In [32]:
data.to_csv("Training_data.csv")

In [33]:
pred_grid.head()

,KDA,ID,Predictions
0,1.66,Aatrox_Top,0.449527
1,1.67,Aatrox_Top,0.451874
2,1.68,Aatrox_Top,0.454219
3,1.69,Aatrox_Top,0.456564
4,1.70,Aatrox_Top,0.458910


In [34]:
pred_grid.to_csv("Prediction_Grid.csv")